In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install streamlit

In [ ]:
!pip install mtranslate

In [ ]:
from PIL import Image
from transformers import BlipProcessor, BlipForConditionalGeneration
import os
import torch  # Add this import

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large").to(device)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
!pip install googletrans==4.0.0-rc1

In [ ]:
# Define the image preprocessing transformations
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Define the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the model and move it to the device
model = models.resnet50(pretrained=True)
model.to(device)
model.eval()  # Set the model to evaluation mode

# Define the tokenizer (if using Hugging Face Transformers)
# tokenizer = AutoTokenizer.from_pretrained("model_name")

# Define the Google Translator
translator = Translator()


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 158MB/s]


In [ ]:
import os
import torch
import torchvision.transforms as transforms
import torchvision.models as models
from PIL import Image
import numpy as np
import re
from googletrans import Translator

# Initialize the translator
translator = Translator()


# Open the output file for writing
# with open(output_file_path, "w") as output_file:
    # Read the train.txt file
merged = []
images_dir = '/content/drive/MyDrive/jyodata/new'
with open('/content/drive/MyDrive/jyodata/ldta.txt', "r", encoding="utf-8") as file:
    train_data = file.read()

    # Split the train data into chunks
    image_chunks = train_data.strip().split("\n\n")

    # Process each chunk
    total_chunks = len(image_chunks)

    for j, chunk in enumerate(image_chunks):
        progress_percentage = (j + 1) / total_chunks * 100
        print(f"Progress: {progress_percentage:.2f}%")

        lines = chunk.strip().split('\n')
        parts = lines[0]

        # Extract image ID using regular expressions
        match = re.search(r'\d+', parts)
        image_id = match.group()

        # Extract annotated text
        annotated_text = "\n".join(lines[1:])

        # Load the image
        image_path = os.path.join(images_dir, f"{image_id}.jpg")
        image = Image.open(image_path).convert('RGB')

        # Conditional image captioning
        text = "a photography of"
        inputs = processor(image, text, return_tensors="pt").to(device)
        out = model.generate(**inputs)
        conditional_caption = processor.decode(out[0], skip_special_tokens=True)

        # Translate conditional caption to Hindi
        conditional_caption_hindi = translator.translate(conditional_caption, src='en', dest='hi').text

        # Unconditional image captioning
        inputs = processor(image, return_tensors="pt").to(device)
        out = model.generate(**inputs)
        unconditional_caption = processor.decode(out[0], skip_special_tokens=True)

        # Translate unconditional caption to Hindi if not None
        if unconditional_caption:
            unconditional_caption_hindi = translator.translate(unconditional_caption, src='en', dest='hi').text
        else:
            unconditional_caption_hindi = "Unable to translate"

        # Print the results

        print("Image ID:", image_id)
        print("Annotated Text:", annotated_text)
        print("Conditional Caption (Hindi):", conditional_caption_hindi)
        print("Unconditional Caption (Hindi):", unconditional_caption_hindi)

        # Create merged text
        merged_text = f"Img_ID:\t{image_id}\n" + annotated_text + "\n" + "<eos>" + "\tB-X\n"
        words = unconditional_caption_hindi.split()
        for i, word in enumerate(words):
            merged_text += f"{word}\tB-X\n"


        merged.append(merged_text+"\n")



# Print completion message
print("All captions processed and saved to output.txt")


Progress: 0.27%


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1141: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Streaming output truncated to the last 5000 lines.
Conditional Caption (Hindi): एक आदमी और एक महिला की एक फोटोग्राफी जो एक इमारत के सामने खड़ी है
Unconditional Caption (Hindi): एक आदमी और एक महिला की तस्वीर एक इमारत के सामने खड़ी है
Progress: 27.99%
Image ID: 401127
Annotated Text: हैदराबाद	B-LOCATION
में	O
ASP-DCP	B-NUMEX
गिरफ्तार,	I-NUMEX
सबूत	B-NUMEX
मिटाने	O
और	O
फोन	O
टैपिंग	O
का	O
मामला	O
Conditional Caption (Hindi): पक्ष में एक बड़े बैज के साथ एक पुलिस कार की एक फोटोग्राफी
Unconditional Caption (Hindi): साइड में बैज के साथ एक पुलिस कार का एक करीबी
Progress: 28.26%
Image ID: 401128
Annotated Text: 'हैदराबाद	B-TIMEX
से	B-LOCATION
सीमांचल	I-NUMEX
में	B-ORGANIZATION
वोट	O
काटने	O
आ	O
रहे	O
हैं',	O
ओवैसी	O
पर	O
तेजस्वी	O
का	B-PERSON
तीखा	O
हमला	B-PERSON
Conditional Caption (Hindi): एक दाढ़ी के साथ एक आदमी की फोटोग्राफी और एक सफेद शर्ट में दाढ़ी के साथ एक आदमी
Unconditional Caption (Hindi): aak aak aak aak aak aak aak aak aak aa
Progress: 28.53%
Image ID: 401129
Annotated Text: कोलकात

In [ ]:
print("Merged data saved to output.txt.")
with open(r"/content/drive/MyDrive/jyodata/newoutput.txt", "w", encoding="utf-8") as file:
    file.writelines(merged)

Merged data saved to output.txt.


In [ ]:
print("Merged data saved to output.txt.")
with open(r"C:\Users\kjnih\Desktop\jyooutput", "w", encoding="utf-8") as file:
    file.writelines(merged)